## parameter search

In [ ]:
# !pip install scikeras
# !pip install keras-tuner --upgrade

In [5]:
import model_helper_Search
import pandas as pd
import numpy as np
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from scikeras.wrappers import KerasClassifier
import keras_tuner
import matplotlib.pyplot as plt
import pickle

tf.random.set_seed(99)

target_outcome = '12months'
max_codes = 150

In [6]:
# # %%time

trainingData, valData, evalData, testData, testDataWales, testDataScotland, vocab_size_clinical, vocab_size_therapy, month_size = model_helper_Search.load_data_split(target_outcome)

#generate X and y
Xt_train, Xt_val, Xt_eval, Xt_test, Xt_testWales, Xt_testScotland, Xs_clin_train, Xs_clin_val, Xs_clin_eval, Xs_clin_test, Xs_clin_testWales, Xs_clin_testScotland, Xs_ther_train, Xs_ther_val, Xs_ther_eval, Xs_ther_test, Xs_ther_testWales, Xs_ther_testScotland, Xm_clin_train, Xm_clin_val, Xm_clin_eval, Xm_clin_test, Xm_clin_testWales, Xm_clin_testScotland, Xm_ther_train, Xm_ther_val, Xm_ther_eval, Xm_ther_test, Xm_ther_testWales, Xm_ther_testScotland, y_train, y_val, y_eval, y_test, y_testWales, y_testScotland = model_helper_Search.generate_X_y_split(trainingData, valData, evalData, testData, testDataWales, testDataScotland, target_outcome)



----------------- Load Data ------------------------------
(94701, 16)
-----------------we passed the heaviest part------------------------------


/rapids/notebooks/host/python/model_helper_Search.py:348: DtypeWarning: Columns (25,26,28,29,32,33,80,84) have mixed types. Specify dtype option on import or set low_memory=False.
  tabularData = pd.read_csv('../FinalData/cleaned_features_01122023.csv')


13
(94701, 16)
(696659, 100)
(94701, 46)
########## Data split, train=England, eval=Scot+Wales ################
Train:  45552
Val:  24403
Eval (internal validation):  11388
Test:  5287
Test - Wales:  3023
Test - Scotland:  2264
############# make sure no data leak between sets #######################
[]
[]
[]
[]
[]
2264
############# Positive and negative groups ratio #######################
0    0.834694
1    0.165306
Name: 12months, dtype: float64
0    0.834692
1    0.165308
Name: 12months, dtype: float64
0    0.834651
1    0.165349
Name: 12months, dtype: float64
0    0.772839
1    0.227161
Name: 12months, dtype: float64
0    0.780681
1    0.219319
Name: 12months, dtype: float64
0    0.762367
1    0.237633
Name: 12months, dtype: float64
############## Generate X (Xt: tabular | Xs: read code sequence | Xm: month visit sequence) and y ####################


In [7]:
training_dataset_ClinTher = Xt_train, Xt_val, Xt_eval, Xs_clin_train, Xs_clin_val, Xs_clin_eval, Xs_ther_train, Xs_ther_val, Xs_ther_eval, Xm_clin_train, Xm_clin_val, Xm_clin_eval, Xm_ther_train, Xm_ther_val, Xm_ther_eval, y_train, y_val, y_eval
test_dataset_ClinTher = Xt_test, Xt_testWales, Xt_testScotland, Xs_clin_test, Xs_clin_testWales, Xs_clin_testScotland, Xs_ther_test, Xs_ther_testWales, Xs_ther_testScotland, Xm_clin_test, Xm_clin_testWales, Xm_clin_testScotland, Xm_ther_test, Xm_ther_testWales, Xm_ther_testScotland, y_test, y_testWales, y_testScotland
pickle.dump(training_dataset_ClinTher, open('../SeqModel/training_dataset_ClinTher.sav', 'wb'))
pickle.dump(test_dataset_ClinTher, open('../SeqModel/test_dataset_ClinTher.sav', 'wb'))

In [ ]:
sets = trainingData, valData, evalData, testData, testDataWales, testDataScotland
pickle.dump(sets, open('../SeqModel/datasets_1year_clinical.sav', 'wb'))

In [ ]:
# n_train = 20000
# n_val = int((20/100)*50000)
# sets_parameterSearch = Xt_train[:n_train], Xt_val[:n_val], Xs_train[:n_train], Xs_val[:n_val], Xm_train[:n_train], Xm_val[:n_val], y_train[:n_train], y_val[:n_val]
sets_parameterSearch = Xt_train, Xt_val, Xt_eval, Xs_train, Xs_val, Xs_eval, Xm_train, Xm_val, Xm_eval, y_train, y_val, y_eval
setsEval_parameterSearch = Xt_test, Xt_testWales, Xt_testScotland, Xs_test, Xs_testWales, Xs_testScotland, Xm_test, Xm_testWales, Xm_testScotland, y_test, y_testWales, y_testScotland
pickle.dump(sets_parameterSearch, open('../SeqModel/sets_1year_therapy.sav', 'wb'))
pickle.dump(setsEval_parameterSearch, open('../SeqModel/sets_eval_1year_therapy.sav', 'wb'))

# sets = pickle.load(open('../SeqModel/sets_search_therapy.sav', 'rb'))

In [ ]:
tuner = model_helper_Search.search_model_parameters('early')

In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = model_helper_Search.MyHyperModel_late()
model = model.build(best_hps[0])

In [ ]:
sets = pickle.load(open('../SeqModel/sets.sav', 'rb'))
Xt_train, Xt_val, Xt_eval, Xt_test, Xt_testWales, Xt_testScotland, Xs_train, Xs_val, Xs_eval, Xs_test, Xs_testWales, Xs_testScotland, Xm_train, Xm_val, Xm_eval, Xm_test, Xm_testWales, Xm_testScotland, y_train, y_val, y_eval, y_test, y_testWales, y_testScotland = sets


In [ ]:
pos_weight = sum(x == 0 for x in y_train)/sum(x == 1 for x in y_train)
class_weight = {0:1, 1:pos_weight}

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
earlyStopping = EarlyStopping(monitor='val_auc', patience=8, verbose=0, mode='max', restore_best_weights=True)
mcp_save = ModelCheckpoint('../SeqModel/seqModel_all_tabSeq.mdl_wts.hdf5', save_best_only=True, monitor='val_auc', mode='min')
# log_dir = "../SeqModel/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit([Xt_train, Xs_train, Xm_train], y_train, validation_data=([Xt_val, Xs_val, Xm_val], y_val), 
                        epochs=100, batch_size=128, 
                            class_weight = class_weight, 
                            callbacks = [earlyStopping, mcp_save])

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
# plt.ylim(0.55,1)
plt.title('model AUC')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
# plt.ylim(0.1, 1.15)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
%%time
#model evaluation
Xts = [Xt_eval, Xt_test, Xt_testWales, Xt_testScotland]
Xss = [Xs_eval, Xs_test, Xs_testWales, Xs_testScotland]
Xms = [Xm_eval, Xm_test, Xm_testWales, Xm_testScotland]
ys = [y_eval, y_test, y_testWales, y_testScotland]
sets = zip(Xts, Xss, Xms, ys)
model_helper_Search.evaluate_model(model, sets)